In [31]:
## only vis

import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

img = sitk.ReadImage(r"D:\\Noha_annotation_converted\\MCF10A\\no apotome raw\\MCF-10A no apotome-01.nii.gz")
arr = sitk.GetArrayFromImage(img)  
print("Shape:", arr.shape)
print("Dtype:", arr.dtype, "Min:", arr.min(), "Max:", arr.max())
z = 30  
arr_norm = (arr - arr.min()) / (arr.max() - arr.min())  # scale 0–1 safely
plt.imshow(arr_norm[z], cmap='gray', vmin=0, vmax=1)
plt.title(f"Slice {z+1} normalized 0–1 (global min–max)")
plt.axis("off")
plt.show()


In [5]:
## only info
import SimpleITK as sitk
import os

def show_nii_info(file_path):
    """
    Show key metadata for a single NIfTI file (.nii or .nii.gz):
      - voxel spacing (in mm)
      - image size (in voxels)
      - origin and direction
    """
    if not os.path.exists(file_path):
        print(f"[ERROR] File not found: {file_path}")
        return

    try:
        img = sitk.ReadImage(file_path)
        spacing = img.GetSpacing()
        size = img.GetSize()
        origin = img.GetOrigin()
        direction = img.GetDirection()

        print(f"\n[INFO] NIfTI file: {os.path.basename(file_path)}")
        print("-" * 60)
        print(f"  Size (voxels):     {size}")
        print(f"  Spacing (mm):      {spacing}")
        print(f"  Spacing (µm):      ({spacing[0]*1000:.4f}, {spacing[1]*1000:.4f}, {spacing[2]*1000:.4f})")
        print(f"  Origin (mm):       {origin}")
        print("  Direction Cosines:")
        print(f"    [{direction[0]:.3f}, {direction[1]:.3f}, {direction[2]:.3f}]")
        print(f"    [{direction[3]:.3f}, {direction[4]:.3f}, {direction[5]:.3f}]")
        print(f"    [{direction[6]:.3f}, {direction[7]:.3f}, {direction[8]:.3f}]")
        print("-" * 60)

    except Exception as e:
        print(f"[ERROR] Failed to read {file_path}: {e}")

# Example usage:
show_nii_info(r"C:\Users\msohaib\Desktop\isotropic\AT1\apotome\AT1_apotome_1_08.nii.gz")



[INFO] NIfTI file: AT1_apotome_1_08.nii.gz
------------------------------------------------------------
  Size (voxels):     (1216, 1028, 76)
  Spacing (mm):      (0.00017250000382773578, 0.00017250000382773578, 0.0010000000474974513)
  Spacing (µm):      (0.1725, 0.1725, 1.0000)
  Origin (mm):       (0.0, 0.0, 0.0)
  Direction Cosines:
    [1.000, 0.000, 0.000]
    [0.000, 1.000, 0.000]
    [0.000, 0.000, 1.000]
------------------------------------------------------------


In [29]:
## only make isotropic
import os
import glob
import SimpleITK as sitk

def make_isotropic_folder(input_path, target_spacing_mm=0.001, interpolator="linear", recursive=False):
    """
    Resample all .nii.gz files in a folder (or one file if a single path is given)
    to isotropic voxel spacing.
    Saves outputs with '_iso.nii.gz' suffix in the same folder.
    """
    # --- if single file provided ---
    if os.path.isfile(input_path) and input_path.lower().endswith(".nii.gz"):
        _resample_single(input_path, target_spacing_mm, interpolator)
        return

    # --- if folder provided ---
    pattern = "**/*.nii.gz" if recursive else "*.nii.gz"
    nii_files = glob.glob(os.path.join(input_path, pattern), recursive=recursive)
    if not nii_files:
        print(f"[WARN] No .nii.gz files found in {input_path}")
        return

    print(f"[INFO] Found {len(nii_files)} file(s) in {input_path}")
    for fp in nii_files:
        _resample_single(fp, target_spacing_mm, interpolator)


def _resample_single(file_path, target_spacing_mm, interpolator):
    """Helper: resample one file and save _iso.nii.gz next to it."""
    try:
        img = sitk.ReadImage(file_path)
        old_spacing = img.GetSpacing()
        old_size = img.GetSize()

        print(f"\n[INFO] {os.path.basename(file_path)}")
        print(f"  Original spacing (mm): {old_spacing}")
        print(f"  Original size:         {old_size}")

        new_spacing = (target_spacing_mm,) * 3
        new_size = [
            int(round(old_spacing[i] * old_size[i] / target_spacing_mm))
            for i in range(3)
        ]

        resampler = sitk.ResampleImageFilter()
        resampler.SetOutputSpacing(new_spacing)
        resampler.SetSize(new_size)
        resampler.SetOutputOrigin(img.GetOrigin())
        resampler.SetOutputDirection(img.GetDirection())
        resampler.SetInterpolator(
            sitk.sitkLinear if interpolator == "linear" else sitk.sitkNearestNeighbor
        )

        img_iso = resampler.Execute(img)

        out_path = os.path.splitext(file_path)[0].replace(".nii", "_iso.nii.gz")
        sitk.WriteImage(img_iso, out_path)

        print(f"  [OK] Saved isotropic file: {out_path}")
        print(f"  New spacing (mm): {img_iso.GetSpacing()}")
        print(f"  New size:         {img_iso.GetSize()}")

    except Exception as e:
        print(f"  [ERROR] {os.path.basename(file_path)}: {e}")



# For a folder:
make_isotropic_folder(r"C:\Users\msohaib\Desktop\isotropic\MCF10A\no_apotome_raw", target_spacing_mm=0.001)


[INFO] Found 3 file(s) in C:\Users\msohaib\Desktop\isotropic\MCF10A\no_apotome_raw

[INFO] MCF_10A_no_apotome_01.nii.gz
  Original spacing (mm): (0.00017250000382773578, 0.00017250000382773578, 0.0010000000474974513)
  Original size:         (1216, 1028, 63)
  [OK] Saved isotropic file: C:\Users\msohaib\Desktop\isotropic\MCF10A\no_apotome_raw\MCF_10A_no_apotome_01_iso.nii.gz
  New spacing (mm): (0.001, 0.001, 0.001)
  New size:         (210, 177, 63)

[INFO] MCF_10A_no_apotome_06.nii.gz
  Original spacing (mm): (0.00017250000382773578, 0.00017250000382773578, 0.0010000000474974513)
  Original size:         (1216, 1028, 80)
  [OK] Saved isotropic file: C:\Users\msohaib\Desktop\isotropic\MCF10A\no_apotome_raw\MCF_10A_no_apotome_06_iso.nii.gz
  New spacing (mm): (0.001, 0.001, 0.001)
  New size:         (210, 177, 80)

[INFO] MCF_10A_no_apotome_16.nii.gz
  Original spacing (mm): (0.00017250000382773578, 0.00017250000382773578, 0.0010000000474974513)
  Original size:         (1216, 1028, 6